# Option A: from CSV (TODO: word - keys one2many issue persisting)

In [33]:
import pandas as pd

In [34]:
DATA_FOLDER = 'data'
CSV_FOLDER = 'WordNet-DictionaryThesaurus'
SYNONYMS_FILENAME = 'WordnetSynonyms.csv'
FREQUENCY_FILENAME = 'word-freq-top5000.csv'
MIN_WORD_LEN = 7

In [35]:
synonyms_path = f'{DATA_FOLDER}/{CSV_FOLDER}/{SYNONYMS_FILENAME}'
frequency_path = f'{DATA_FOLDER}/{CSV_FOLDER}/{FREQUENCY_FILENAME}'
df_synonyms = pd.read_csv(synonyms_path)
df_frequency = pd.read_csv(frequency_path)

In [36]:
def flatten(lst):
    """
    Example:
    lst = [1, 2, [3, [4, 5]], [6, 7, 8], 9, [10, 11]]\n
    print(flatten(lst))
    >>> [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    """
    result = []
    for item in lst:
        if isinstance(item, list):
            result.extend(flatten(item))
        else:
            result.append(item)
    return result

In [37]:
frequent_words = [word for word in df_frequency.Word if len(word) >= MIN_WORD_LEN]
df_frequent_synonyms = df_synonyms[df_synonyms['Word'].isin(frequent_words)]

frequent_synonyms_dict = {}
for index, row in df_frequent_synonyms.iterrows():
    word = row['Word']
    synonyms = row['Synonyms'].split(';')
    synonyms = [word.split('|') for word in synonyms]
    synonyms = flatten(synonyms)
    if word not in frequent_synonyms_dict:
        frequent_synonyms_dict[word] = synonyms
    else:
        frequent_synonyms_dict[word].extend(synonyms)

word = 'agreement'
synonyms = frequent_synonyms_dict['agreement']
word, synonyms



('agreement',
 ['understanding', 'correspondence', 'accord', 'arrangement', 'concord'])

# Option B: from .jsonl

In [38]:
import json

In [40]:
DATA_FOLDER = 'data'
SYNONYMS_FILENAME = 'en_thesaurus.jsonl'
FREQUENCY_FILENAME = 'unigram_freq.csv'
MIN_WORD_LEN = 7
FREQUENCY_FLOOR = 5000

In [41]:
synonyms_path = f'{DATA_FOLDER}/{SYNONYMS_FILENAME}'
frequency_path = f'{DATA_FOLDER}/{FREQUENCY_FILENAME}'
synonyms_entry_list = [json.loads(entry) for entry in open(synonyms_path)]
df_frequency = pd.read_csv(frequency_path)

In [42]:
synonyms_dict = {}
for n, entry in enumerate(synonyms_entry_list):
    word = entry['word']
    pos = entry['pos']
    synonyms = entry['synonyms']
    if word in synonyms_dict:
        synonyms_dict[word]['synonyms'].extend(synonyms)
        synonyms_dict[word]['pos'].append(pos)
        synonyms_dict[word]['pos'] = list(set(synonyms_dict[word]['pos']))
    else:
        synonyms_dict[word] = {
            'pos': [pos],
            'synonyms': synonyms
        }
    if not n % 10000:
        print('.', end='')

.................

In [43]:
frequent_words = [word for word in df_frequency[:FREQUENCY_FLOOR]['word'].astype(str) if len(word) >= MIN_WORD_LEN]
frequent_synonyms = [word for word in frequent_words if word in synonyms_dict]

In [44]:
word = frequent_synonyms[138]
synonyms = synonyms_dict[word]['synonyms']
word, synonyms

('agreement',
 ['accord', 'arrangement', 'concord', 'correspondence', 'understanding'])